# 1- Imports (Source Code + Libraries)

In [8]:
import pickle
import numpy as np
import pandas as pd

import os

In [9]:
# adding source code parent directory
import sys
sys.path.append('..\\Src')

### 1.1 Data Preprocessing Modules

In [25]:
from preprocessing.data_prep import read_data
from preprocessing.data_clean import clean_data2, clean_text
from preprocessing.feature_extract import create_vectorizer

### 1.2 Search Engine Module

In [11]:
from search.search_engine import Search_Engine

### 1.3 utils

In [12]:
from utils.modify_stop_words import modify_sw

# 2- Preparation for SE

In [6]:
# variables set
data_path = "../Data/husna.json"
cols = ['text_clean', 'summary_clean', 'title_clean', 'tags']
cols_weights = {'text_clean': 0.5, 'summary_clean': 0.15, 'title_clean': 0.2, 'tags': 0.1}

In [7]:
# prepare stop words (use when list.csv is not available)
# modify_sw(path='../Data/stop_words/stop_words_list.txt')

In [13]:
# import -> clean data
docs_df = read_data(data_path)
docs_df_cleaned = clean_data2(docs_df)    

In [14]:
docs_df.shape

(6366, 11)

In [15]:
vectorizers = {}

for col in cols[:-1]:
    # create vectorizer for each column
    vectorizers[col] = create_vectorizer(docs_df_cleaned, from_feature=col)
    
    # saving vectorizers for SE preprocessing
    vectorizer_to_save = vectorizers[col]
    if vectorizer_to_save != None:
        pickle.dump(vectorizer_to_save, open(f"../Results/saved_vectorizers/{col}_tfidf.pickle", "wb"))
        
# saving cleaned data
docs_df_cleaned_SE = docs_df_cleaned[['_id', 'url', 'title', 'text', 'published_at', 'article_type', 'tags', 'text_clean', 'summary_clean', 
                                              'title_clean']]
docs_df_cleaned_SE.to_csv('../Data/processed/SE_data1.csv', index=False, encoding='utf-8-sig')

# 3- using SE

In [27]:
!python -m pip install elasticsearch

^C


In [21]:
SE_df = pd.read_csv('../Data/processed/SE_data1.csv', keep_default_na=False, encoding='utf-8-sig')

In [22]:
SE_df

,_id,url,title,text,published_at,article_type,tags,text_clean,summary_clean,title_clean
0,633fdfd13ffae8229d05cb33,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,التربية: تحويل 42 مدرسة إلى نظام الفترتين واست...,أكدت أمين عام وزارة التربية والتعليم للشؤون ...,2022-10-06 07:33:00,News,"['التربية والتعليم', 'وزارة التربية والتعليم']",اكد امين عام وزاره تربيه تعليم شان مالي اداري ...,,تربيه تحويل 42 مدرسه الي نظام فتره استءجار 1...
1,633fdfd13ffae8229d05cb34,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,تكريما للمعلمين زيادة منح أبناء المعلمين 550 م...,احتفلت وزارة التربية والتعليم بيوم المعلم بت...,2022-10-05 11:46:00,News,"['مكرمة أبناء المعلمين', 'وزارة التربية والتعل...",احتفل وزاره تربيه تعليم يوم معلم تكريم نخبه مع...,,تكريم معلم زياده منح ابناء معلم 550 مقعد اضافي...
2,633fdfd13ffae8229d05cb35,https://husna.fm/%D9%85%D9%84%D9%81%D8%A7%D8%A...,الغاز الروسي يضع أوروبا خلال الشتاء المقبل أما...,يشهد العالم أول أزمة طاقة عالمية حقيقية في الت...,2022-08-22 07:17:00,News,"['الغاز الروسي', 'أوروبا', 'أزمة الطاقة']",شهد عالم اول ازمه طاقه عالم حقيقي تاريخ . تنام...,,غاز روسي وضع اوربا خلال شتاء مقبل امام اختبار ...
3,633fdfd13ffae8229d05cb36,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,تفاصيل دوام المدارس بعد قرار تثبيت التوقيت الصيفي,كشفت أمين عام وزارة التربية والتعليم للشؤون ال...,2022-10-06 05:29:00,News,"['التربية والتعليم', 'التوقيت الصيفي', 'المدارس']",كشف امين عام وزاره تربيه تعليم شان مالي اداري ...,,تفصيل دوام مدرسه قرار تثبيت توقيت صيفي
4,633fdfd13ffae8229d05cb37,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,العمل: عطلة المولد النبوي الشريف تشمل القطاع ا...,أكدت وزارة العمل أن العطل الرسمية تكون مأجور...,2022-10-06 07:46:00,News,"['وزارة العمل', 'المولد النبوي']",اكد وزاره عمل ان عطل رسمي ماجور عامل مءسسه قطا...,,عمل عطله مولد نبوي شريف شمل قطاع خاص
...,...,...,...,...,...,...,...,...,...,...
5079,633ff10f322e7883822052b3,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,الديمقراطية والتقاء الخصوم,نتحدث هنا عن القوى المؤثرة على الأرض العربية ا...,2022-09-20 21:00:00,Blog,['الديموقراطية'],تحدث قوه مءثر علي ارض عربي اصبح سبخ مالح بيءه ...,,ديموقراطي التقاء خصم
5080,633ff10f322e7883822052b4,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,علموا أولادكم ألوان الطيف السبعة ومعانيها,سبحان من خلق الكون وأبدع في تصوريه، فجعل السما...,2022-09-21 12:18:00,Blog,"['ألوان الطيف السبعة', 'الشذوذ الجنسي']",خلق كون بدع صور ، سماء درجه ، ارض منزل ، ايه ا...,,ولد لون طيف معني
5081,633ff10f322e7883822052b7,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,تدين الخلاص الفردي وفرص النجاح,حالة من الصراع الصفري بين الأنظمة والإسلام الس...,2022-10-02 21:00:00,Blog,"['الإسلام', 'الإسلام السياسي']",حال صراع صفر ي نظام اسلام سياسي تولد ربيع عربي...,,ادان خلاص فردي فرصه نجاح
5082,633ff110322e7883822052b9,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,السر وراء الانخفاض الملحوظ في أسعار الأجهزة ال...,جدل يسود الشارع الأردني مجددا إثر التخفيضات ال...,2022-10-04 21:00:00,Blog,['تخفيضات'],جدل ساد شارع اردني مجدد اثر تخفيض كبير علي سعر...,,سر وراء انخفاض ملحوظ سعر جهاز كهرباءي


In [16]:
SE_ob = Search_Engine(cols, cols_weights, df=SE_df)

In [17]:
SE_ob.get_saved_vectorizers(rel_dir='../Results/saved_vectorizers')

C:\Users\modaj\OneDrive\Documents\Personal\Jobs\SHAI\intern - task 3\SEA_venv\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\modaj\OneDrive\Documents\Personal\Jobs\SHAI\intern - task 3\SEA_venv\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [18]:
results = SE_ob.overall_search('مولد النبي')

C:\Users\modaj\OneDrive\Documents\Personal\Jobs\SHAI\intern - task 3\SEA_venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\modaj\OneDrive\Documents\Personal\Jobs\SHAI\intern - task 3\Notebooks\..\Src\search\search_engine.py:59: RuntimeWarning: invalid value encountered in double_scalars
  sim[i] = np.dot(data.loc[:, i].values, q_vec) / np.linalg.norm(data.loc[:, i]) * np.linalg.norm(q_vec)
C:\Users\modaj\OneDrive\Documents\Personal\Jobs\SHAI\intern - task 3\SEA_venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [51]:
np.array(top10_results, dtype="int32")[:, 0]

4600

In [69]:
ids = np.array(top10_results, dtype="int32")[:, 0]
urls_temp = list(SE_df.loc[ids, :]['url'])
title_temp = list(SE_df.loc[ids, :]['title'])
text_temp = list(SE_df.loc[ids, :]['text'])

In [26]:
q = 'التعليم'
clean_text(q)

'التعليم'

In [68]:
top10_results = sorted(results[0], key=lambda x: x[1], reverse=True)[:10]
SE_time = results[1]
SE_score = results[2]
ids = np.array(top10_results, dtype="int32")[:, 0]
urls = list(SE_df.loc[ids, :]['url'])
titles = list(SE_df.loc[ids, :]['title'])
texts = list(SE_df.loc[ids, :]['text'])

print(f'-> Top 10 Results:')
print('-' * 100, end='\n\n')
for i, (t10r_id, t10r_score) in enumerate(top10_results):
    print('Document ID {:d} with a score of {:.3f}'.format(int(t10r_id), t10r_score))
    print(f"link: {urls[i]}", end='\n\n')
    print(f"title: {titles[i]}")
    print(f"description: {texts[i][:100]}")
    print('-' * 50, end='\n\n')
print()
print('Search Time Duration: {:.2f} Seconds'.format(SE_time * 10 **-3))
print('Search (uniqueness/frequency) fraction: {:.3f}'.format(SE_score))

-> Top 10 Results:
----------------------------------------------------------------------------------------------------

Document ID 4600 with a score of 0.100
link: https://husna.fm/%D8%B9%D8%B1%D8%A8%D9%8A-%D9%88-%D8%AF%D9%88%D9%84%D9%8A/%D8%A7%D9%84%D9%86%D8%A8%D9%8A-%D9%85%D8%AD%D9%85%D8%AF-%28%EF%B7%BA%29

title: خان يطالب بتجريم إهانة النبي محمد أسوة بإنكار المحرقة اليهودية
description: شدد  رئيس الوزراء الباكستاني  عمران خان في تغريدات  له على حسابه الخاص بموقع (تويتر) أن على الحكومات
--------------------------------------------------

Document ID 1697 with a score of 0.077
link: https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D9%84%D9%8A%D9%84%D8%A9-%D8%A7%D9%84%D8%A5%D8%B3%D8%B1%D8%A7%D8%A1-%D9%88%D8%A7%D9%84%D9%85%D8%B9%D8%B1%D8%A7%D8%AC-%D8%AA%D8%A8%D8%AB-%D9%81%D9%8A-%D8%A7%D9%84%D9%86%D9%81%D9%88%D8%B3-%D8%A7%D9%84%D8%B5%D8%A8%D8%B1-%D8%B9%D9%84%D9%89-%D8%AA%D8%AC%D8%A7%D9%88%D8%B2-%D8%A7%D9%84%D8%B4%D8%AF%D8%A7%D8%A6%D8%AF

title: ليلة الإسراء والمعراج تبث في النفوس الصبر على 